In [1]:
import timeit
start = timeit.default_timer() # To check the running time
import pandas
import numpy
from docx import Document
import requests, uuid, re
import concurrent.futures

In [28]:
name = '6.12 Procedures of Winding up of a Company' # Name of the document
doc = Document(name + '.docx')

In [ ]:
#Extracting each line for translation from the doc file
lis = []
for p in doc.paragraphs:
    oho = p.text
    lis.append(oho.strip())

In [29]:
def translator(input_edited):
    # Micrsoft Azure API key and endpoint
    key = "4b2m62c85okcs3j4o1pp2mwi1h8j3d53" #This is a random value (Not my key)
    endpoint = "https://api.cognitive.microsofttranslator.com"

    # location, also known as region.
    #It can be found in the Azure portal on the Keys and Endpoint page.
    location = "eastus"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['ne']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location is required if using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    # Pass the object in body.
    body = [{
        'text': input_edited
    }]
    
    # Sending request and geting response
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    
    return response[0]

In [30]:
#Extracting the translated text from the response
def translated_extraction(text):
    translated = translator(text)
    
    text_dic = translated["translations"]
    text = text_dic[0]
    translated_text = text["text"]
    
    return translated_text

In [32]:
# Creating a new document for traslated sentences
new_doc = Document()

# Add a paragraph to the document
p = new_doc.add_paragraph()

# Add some formatting to the paragraph
p.paragraph_format.line_spacing = 1.5
p.paragraph_format.space_after = 1

#Function to add text to the document
def add_text(text):
    run = p.add_run(text)
    run.add_break()

In [33]:
# sending each sentence for translation and adding it to the new document
i = 0
with concurrent.futures.ThreadPoolExecutor() as executor: 
    for result in executor.map(translated_extraction, lis):
        if not lis[i] or re.search("^\s*$", lis[i]):
            text = lis[i]
            new_doc.add_paragraph()
            
        else: 
            text = lis[i], f"({result})"
            #print(lis[i], f"({result})")

        #Inserting the translated text in new doc    
        add_text(text)
        i = i + 1

{'translations': [{'text': 'निरन्तर नाफा', 'to': 'ne'}]}
{'translations': [{'text': 'अनिवार्य परिसमापन', 'to': 'ne'}]}
{'translations': [{'text': 'निरन्तर नोक्सानी', 'to': 'ne'}]}
{'translations': [{'text': 'निरन्तर उत्पादन', 'to': 'ne'}]}
{'translations': [{'text': 'स्वैच्छिक परिसमापन', 'to': 'ne'}]}
{'translations': [{'text': 'प्रविधिको नियमित अद्यावधिक', 'to': 'ne'}]}
{'translations': [{'text': '', 'to': 'ne'}]}
{'translations': [{'text': '- 6.12 एक कंपनी को घुमाने की प्रक्रियाएं', 'to': 'ne'}]}
{'translations': [{'text': '', 'to': 'ne'}]}
{'translations': [{'text': '२. यदि कुनै कम्पनी दिवालिया भयो भने त्यो __________ जान्छ ।', 'to': 'ne'}]}
{'translations': [{'text': 'निम्न मध्ये कुन अवस्थामा कम्पनीले खारेजीको सामना गर्न सक्छ?', 'to': 'ne'}]}
{'translations': [{'text': 'प्राकृतिक परिसमापन', 'to': 'ne'}]}
{'translations': [{'text': '', 'to': 'ne'}]}
{'translations': [{'text': '', 'to': 'ne'}]}
{'translations': [{'text': 'अप्राकृतिक परिसमापन', 'to': 'ne'}]}
{'translations': [{'text':

In [34]:
#styling the new doc
from docx.shared import Pt

style = new_doc.styles['Normal']
font = style.font
font.name = 'Proxima Nova'
font.size = Pt(10)

#Save the new doc
new_doc.save(name + '_translated.docx')

In [35]:
stop = timeit.default_timer()
print('total_time:', stop - start)

total_time: 157.31264519994147
